In [1]:
from qiskit.transpiler import CouplingMap, PassManager
from qiskit.transpiler.passes import SabreLayout, SabreSwap
from qiskit.transpiler.passes import FullAncillaAllocation, EnlargeWithAncilla, ApplyLayout
from qiskit.converters import circuit_to_dag, dag_to_circuit
from utils import file_to_coupling_map, directory_to_circuits
import random
import time
import os
import numpy as np


In [2]:
seed = 42
random.seed(seed)

cmap_name= "heavy_hex_127.txt"
cmap = file_to_coupling_map(f"coupling_maps/{cmap_name}")
qc_name = "qft_107"
heuristic = "decay"
curr_seed = 42
qc_list = directory_to_circuits(f"circuits/{qc_name}")

pass_trials_list = sorted(set(np.logspace(0, 3, num=10, dtype=int)))


# CSV file to store the results
csv_path = f"results/trials_{qc_name}_10.csv"
# Columns are: "swap_count", "depth", "runtime", "num_qubits", "heuristic", "cmap", "trials", "seed"
with open(csv_path, "w") as f:
    f.write("swap_count,depth,runtime,num_qubits,heuristic,cmap,trials,seed\n")




In [3]:
# Run the circuits
for qc in qc_list:
    num_qubits = qc.num_qubits
    print(f"    Running {qc.name} ")
    # Run the circuits for different number of trials
    for pass_trials in pass_trials_list:
        print(f"Running {pass_trials} trials")
        # Create the sabre pass
        sabre_pass = SabreLayout(cmap, seed=curr_seed, max_iterations=4, swap_trials=pass_trials, layout_trials=pass_trials )
        dag = circuit_to_dag(qc)
        
        # Execute the pass manager
        start = time.time()
        dag_tr = sabre_pass.run(dag)
        end = time.time()
        # Collect the metrics for this trial
        runtime = end - start

        qc_tr = dag_to_circuit(dag_tr)
        swap_count = qc_tr.count_ops().get("swap", 0)
        depth = qc_tr.depth(lambda x: x.operation.num_qubits == 2)

        # Write the results to the CSV file
        # Columns are: "swap_count", "depth", "runtime", "num_qubits", "heuristic", "cmap", "trials", "seed"
        with open(csv_path, "a") as f:
            f.write(f"{swap_count},{depth},{runtime},{num_qubits},{heuristic},{cmap_name},{pass_trials},{curr_seed}\n")
        print(f"            Swap count: {swap_count}, Depth: {depth}, Runtime: {runtime}")

    

    Running qft_107.qasm 
Running 1 trials
            Swap count: 10499, Depth: 2758, Runtime: 5.672595739364624
Running 2 trials
            Swap count: 9248, Depth: 2451, Runtime: 6.688628911972046
Running 4 trials
            Swap count: 7497, Depth: 1649, Runtime: 8.21804404258728
Running 10 trials
            Swap count: 7497, Depth: 1649, Runtime: 19.160216808319092
Running 21 trials
            Swap count: 7303, Depth: 1406, Runtime: 72.1675808429718
Running 46 trials
            Swap count: 7303, Depth: 1406, Runtime: 286.79152512550354
Running 100 trials
            Swap count: 7303, Depth: 1406, Runtime: 1234.4526698589325
Running 215 trials
            Swap count: 7194, Depth: 1527, Runtime: 5600.131521940231
Running 464 trials
